In [1]:
from dataloader import *
from  config import *
from data_processor import preprocess_data

Using TensorFlow backend.


In [2]:
data = DataLoader(config_mixed_flickr8k_flickr8k_n)
# print(config_mixed_flickr8k_flickr8k)

In [3]:
print(preprocess_data(data))

{'1056338697_4f7d7ce270': ['START blondynka w niebieskiej koszuli wydaje się czekać na przejażdżkę STOP', 'START blondynka jest na ulicy i zaprasza taksówkę STOP', 'START kobieta sygnalizuje ruch drogowy jak widać od tyłu STOP', 'START kobieta z blond włosami w niebieskim topie z rurką macha na poboczu ulicy STOP', 'START kobieta w niebieskiej sukience wyciąga rękę na nadjeżdżający ruch uliczny STOP'], '106490881_5a2dd9b7bd': ['START chłopiec w niebieskich spodenkach kąpielowych na plaży STOP', 'START chłopiec uśmiecha się do kamery na plaży STOP', 'START młody chłopiec w kąpielówkach idzie z wyciągniętymi rękami po plaży STOP', 'START dzieci bawiące się na plaży STOP', 'START chłopiec bawi się na brzegu oceanu STOP'], '1082379191_ec1e53f996': ['START dama i mężczyzna bez koszuli siedzą na doku STOP', 'START mężczyzna i kobieta siedzą razem na doku STOP', 'START mężczyzna i kobieta siedzący na doku STOP', 'START mężczyzna i kobieta siedzący na pokładzie obok jeziora STOP', 'START mężcz

NameError: name 'train_images' is not defined

In [ ]:
len(data.test["train_images_mapping_original"])

In [ ]:
len(data.train["test_images_mapping_original"])

In [ ]:
len(data.test["train_images_mapping_original"])

In [ ]:
len(data.train["train_images_mapping_original"])

In [ ]:
len(data.test["all_captions"])

In [ ]:
data.train["all_captions"]

In [ ]:
len(data.test["train_captions_mapping_original"])

In [ ]:
len(data.train["train_captions_mapping_original"])

In [ ]:
len(data.test["test_captions_mapping_original"])

In [ ]:
len(data.train["test_captions_mapping_original"])

In [ ]:
model=ModelImpl(data)

In [ ]:
model.train()

In [ ]:
model.evaluate()

In [ ]:
generate_report(data.config["results_directory"])